In [1]:
!pip install wikipedia

import wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=d9bb71a3795ccff4ea91c47484e4100b5e6eb872e11a7d27d53c1a970ecec70c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [34]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding,SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import random
import gensim
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data]   Package omw-1.4 is already up-to-date!
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
replace /kaggle/working/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  /kaggle/working/nltk_data/corpora/omw-1.4.zip
replace /kaggle/working/nltk_data/corpora/omw-1.4/fin/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
# Fetch entire article
full_article1 = wikipedia.page("Artificial Intelligence").content

In [4]:
# Fetch entire article
full_article2 = wikipedia.page("neural network").content

In [5]:
# Fetch entire article
full_article3 = wikipedia.page("Deep learning").content

In [6]:
documents = [[full_article1], [full_article2], [full_article3]]
documents

[['Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, of

In [7]:
def preprocess(documents):
    processed_documents = []
    for document in documents:
        processed_doc = str(document)
        processed_doc = re.sub(r'[^a-zA-Z\s]', '', processed_doc)
        processed_doc = processed_doc.lower()
        
        sentaces_minus_sw = []
        stop_words = stopwords.words('english')
        processed_doc = processed_doc.split()
        processed_doc = [sentaces_minus_sw.append(word) for word in processed_doc if word not in stop_words]
        processed_doc = ' '.join(sentaces_minus_sw)
        
        lemmatizer = WordNetLemmatizer()
        processed_doc = processed_doc.split()
        processed_doc = [lemmatizer.lemmatize(w) for w in processed_doc]
        processed_doc = ' '.join(processed_doc)

        processed_documents.append(processed_doc)

    return processed_documents

In [8]:
processed_docs = preprocess(documents)

In [9]:
processed_docs

['artificial intelligence ai broadest sense intelligence exhibited machine particularly computer system field research computer science develops study method software enable machine perceive environment us learning intelligence take action maximize chance achieving defined goal machine may called aisnai technology widely used throughout industry government science highprofile application include advanced web search engine eg google search recommendation system used youtube amazon netflix interacting via human speech eg google assistant siri alexa autonomous vehicle eg waymo generative creative tool eg chatgpt ai art superhuman play analysis strategy game eg chess go however many ai application perceived ai lot cutting edge ai filtered general application often without called ai something becomes useful enough common enough labeled ai anymorenalan turing first person conduct substantial research field called machine intelligence artificial intelligence founded academic discipline field 

In [10]:
chars = sorted(list(set("".join(processed_docs))))
char_to_index = {char: idx for idx, char in enumerate(chars)}
index_to_char = {idx: char for idx, char in enumerate(chars)}
num_chars = len(chars)

In [11]:
max_sequence_len = 1000

def prepare_sequences(text):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - max_sequence_len, 1):
        seq = text[i:i + max_sequence_len]
        next_char = text[i + max_sequence_len]
        sequences.append([char_to_index[char] for char in seq])
        next_chars.append(char_to_index[next_char])
    return sequences, next_chars

# Prepare sequences for each document
chars_sequences = []
next_chars_list = []
for doc in processed_docs:
    sequences, next_chars = prepare_sequences(doc)
    chars_sequences.extend(sequences)
    next_chars_list.extend(next_chars)

In [12]:
sequences_padded = pad_sequences(chars_sequences, maxlen=max_sequence_len, padding='pre')
X = np.array(sequences_padded)
y = to_categorical(next_chars_list, num_classes=num_chars)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(57879, 1000)
(38587, 1000)
(57879, 27)
(38587, 27)


In [14]:
# Define the RNN model
char_model1 = Sequential([
    Embedding(input_dim=num_chars, output_dim=100, input_length=max_sequence_len),
    SimpleRNN(100),
    Dense(num_chars, activation='softmax')
])
char_model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
char_model1.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
char_model1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
   2/1809 ━━━━━━━━━━━━━━━━━━━━ 2:31 84ms/step - accuracy: 0.0391 - loss: 3.3039 

I0000 00:00:1715983790.394696     128 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1809/1809 ━━━━━━━━━━━━━━━━━━━━ 159s 86ms/step - accuracy: 0.1126 - loss: 3.0184 - val_accuracy: 0.1852 - val_loss: 2.7083
Epoch 2/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 84ms/step - accuracy: 0.2091 - loss: 2.6181 - val_accuracy: 0.2744 - val_loss: 2.4252
Epoch 3/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.2868 - loss: 2.3588 - val_accuracy: 0.3276 - val_loss: 2.2353
Epoch 4/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.3462 - loss: 2.1682 - val_accuracy: 0.3957 - val_loss: 2.0261
Epoch 5/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.4163 - loss: 1.9523 - val_accuracy: 0.4425 - val_loss: 1.8838
Epoch 6/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.4600 - loss: 1.8115 - val_accuracy: 0.4773 - val_loss: 1.7800
Epoch 7/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.4954 - loss: 1.7067 - val_accuracy: 0.4960 - val_loss: 1.7190
Epoch 8/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 153s 85ms/step - accuracy: 0.5145 - lo

In [16]:
loss, accuracy = char_model1.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy*100)

1206/1206 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.5297 - loss: 1.6132
Test Accuracy: 52.688729763031006


In [17]:
# Define the RNN model
char_model2 = Sequential([
    Embedding(input_dim=num_chars, output_dim=300, input_length=max_sequence_len),
    SimpleRNN(128),
    Dense(num_chars, activation='softmax')
])
char_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
char_model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
char_model2.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
   1/1809 ━━━━━━━━━━━━━━━━━━━━ 1:31:53 3s/step - accuracy: 0.0312 - loss: 3.3317

W0000 00:00:1715985351.499658     129 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1809/1809 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.2603 - loss: 2.4952

W0000 00:00:1715985529.087322     128 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1809/1809 ━━━━━━━━━━━━━━━━━━━━ 223s 122ms/step - accuracy: 0.2603 - loss: 2.4951 - val_accuracy: 0.4008 - val_loss: 2.0315
Epoch 2/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 217s 120ms/step - accuracy: 0.4215 - loss: 1.9463 - val_accuracy: 0.4649 - val_loss: 1.8131
Epoch 3/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 217s 120ms/step - accuracy: 0.4871 - loss: 1.7253 - val_accuracy: 0.5013 - val_loss: 1.7046
Epoch 4/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 216s 120ms/step - accuracy: 0.5182 - loss: 1.6148 - val_accuracy: 0.5213 - val_loss: 1.6435
Epoch 5/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 217s 120ms/step - accuracy: 0.5417 - loss: 1.5360 - val_accuracy: 0.5340 - val_loss: 1.5935
Epoch 6/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 216s 120ms/step - accuracy: 0.5547 - loss: 1.4840 - val_accuracy: 0.5348 - val_loss: 1.5834
Epoch 7/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 216s 120ms/step - accuracy: 0.5705 - loss: 1.4413 - val_accuracy: 0.5484 - val_loss: 1.5480
Epoch 8/10
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 216s 120ms/step - accuracy: 0.5

In [19]:
loss, accuracy = char_model2.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy*100)

1206/1206 ━━━━━━━━━━━━━━━━━━━━ 43s 36ms/step - accuracy: 0.5587 - loss: 1.5107
Test Accuracy: 55.86337447166443


In [31]:
def get_random_start_char(processed_docs):
    all_text = ''.join(processed_docs)  
    return random.choice(all_text)  # Randomly select one character

In [30]:
def predict_next_chars_single(model, start_char, num_prediction_steps, char_to_index, index_to_char, max_sequence_len):
    """
    Predict the next characters starting from a single character.

    Parameters:
    model -- Trained RNN model.
    start_char -- The initial character to start the prediction from.
    num_prediction_steps -- Number of characters to predict.
    char_to_index -- Dictionary mapping characters to their index.
    index_to_char -- Dictionary mapping index to their characters.
    max_sequence_len -- The maximum length of the sequence used for prediction.

    Returns:
    predicted_text -- The initial character followed by the predicted characters.
    """
    # Initialize the sequence with the start character
    current_sequence = start_char
    predicted_text = start_char

    for _ in range(num_prediction_steps):
        # Convert the current sequence to indices
        sequence_indices = [char_to_index[char] for char in current_sequence[-max_sequence_len:]]
        
        # Pad the sequence to the required length
        padded_sequence_indices = pad_sequences([sequence_indices], maxlen=max_sequence_len, padding='pre')
        
        # Predict the next character index
        predicted_index = np.argmax(model.predict(padded_sequence_indices), axis=-1)[0]
        
        # Convert the index back to a character
        predicted_char = index_to_char[predicted_index]
        
        # Append the predicted character to the predicted text
        predicted_text += predicted_char
        
        # Update the current sequence with the new character
        current_sequence += predicted_char

    return predicted_text

In [38]:
start_char = get_random_start_char(processed_docs)
print(f"Randomly selected starting character: '{start_char}'")

Randomly selected starting character: 'a'


In [39]:
# Predict the next 10 characters starting from the randomly selected character
predicted_text = predict_next_chars_single(char_model2, start_char, 10, char_to_index, index_to_char, max_sequence_len)

print(f"Starting with '{start_char}', predicted text: {predicted_text}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Starting with 'a', predicted text: ai research
